In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

sns.set_style('whitegrid')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
pd.plotting.register_matplotlib_converters()

In [ ]:
data = pd.read_csv('./HalfHourly_Data.csv', index_col=0)


In [ ]:
VIC_data = data[['VIC_PRICE','VIC_DEMAND', 'VIC_NETINTERCHANGE', 'VIC_BATTERY', 'VIC_BIOMASS',
       'VIC_BLACK_COAL', 'VIC_BROWN_COAL', 'VIC_DISTILLATE', 'VIC_GAS_CCGT',
       'VIC_GAS_OCGT', 'VIC_GAS_RECIP', 'VIC_GAS_STEAM', 'VIC_HYDRO',
       'VIC_PUMPS', 'VIC_SOLAR', 'VIC_WIND','VIC_ROOFTOP_SOLAR']]

In [ ]:
VIC_data.fillna(0, inplace=True)

In [ ]:
VIC_data['VIC_Temp'] = data['VIC_TEMPERATURE']

In [ ]:
plt.scatter(data[data['VIC_PRICE']<500]["VIC_PRICE"],data[data['VIC_PRICE']<500]['VIC_TEMPERATURE'],alpha=0.02)

In [ ]:
plt.scatter(data[data['VIC_PRICE']<500]["VIC_PRICE"],data[data['VIC_PRICE']<500]['VIC_DEMAND'],alpha=0.01)

In [ ]:
plt.scatter(data['VIC_DEMAND'],data['VIC_TEMPERATURE'],alpha=0.01)

In [ ]:
from fbprophet import Prophet, diagnostics


pd.set_option('use_inf_as_na', True)

VIC_data['y'] = np.log1p(VIC_data['VIC_PRICE'])
VIC_data['ds'] = VIC_data.index

In [ ]:
m = Prophet()
m.add_regressor('VIC_DEMAND', mode='multiplicative')
m.add_regressor('VIC_NETINTERCHANGE', mode='multiplicative')
m.add_regressor('VIC_WIND', mode='multiplicative')
m.add_regressor('VIC_HYDRO', mode='multiplicative')
m.add_regressor('VIC_ROOFTOP_SOLAR', mode='multiplicative')
m.add_regressor('VIC_SOLAR', mode='multiplicative')
m.add_regressor('VIC_GAS_OCGT', mode='multiplicative')
m.add_regressor('VIC_BROWN_COAL', mode='multiplicative')
m.add_regressor('VIC_GAS_STEAM', mode='multiplicative')

m.fit(VIC_data)


In [ ]:

#future = m.make_future_dataframe(periods=480)
# future_forecast = m.predict(future)
# m.plot(future)
forecast = m.predict(VIC_data)

In [ ]:
m.plot(forecast)

In [ ]:
m2 = Prophet()

forecast2 = m2.fit(VIC_data).predict(VIC_data)
m2.plot(forecast2)

In [ ]:
m.plot_components(forecast);

In [ ]:
m2.plot_components(forecast2);

In [ ]:
VIC_data.VIC_PRICE.plot()
np.exp(forecast['yhat']).plot()
np.exp(forecast2['yhat']).plot(figsize=(20,5))
plt.ylim(0,400)


In [ ]:
forecast = forecast.set_index('ds')


In [ ]:

VIC_forecast_ohlc = np.exp(forecast['yhat']).resample('3H').ohlc()

In [ ]:
Forecast_price =  np.exp((forecast['yhat'].resample('D').mean()))


In [ ]:
pd.plotting.register_matplotlib_converters()
Forecast_price.plot()


## Running the model on NSW Data

In order to test that the modelling method is not exclusive to the Victorian Market, it was tested on NSW data as well.

In [ ]:
NSW_data = data[['NSW_DEMAND', 'NSW_NETINTERCHANGE', 'NSW_BATTERY',
       'NSW_BIOMASS', 'NSW_BLACK_COAL', 'NSW_BROWN_COAL', 'NSW_DISTILLATE',
       'NSW_GAS_CCGT', 'NSW_GAS_OCGT', 'NSW_GAS_RECIP', 'NSW_GAS_STEAM',
       'NSW_HYDRO', 'NSW_PUMPS', 'NSW_SOLAR', 'NSW_WIND', 'NSW_ROOFTOP_SOLAR', 'NSW_TEMPERATURE']]
NSW_data['y'] = np.log1p(data['NSW_PRICE'])
NSW_data['ds'] = data.index
NSW_data.fillna(0, inplace=True)


In [ ]:
m_nsw = Prophet()
m_nsw.add_regressor('NSW_DEMAND', mode='additive')
m_nsw.add_regressor('NSW_NETINTERCHANGE', mode='additive')
m_nsw.add_regressor('NSW_BLACK_COAL', mode='additive')
m_nsw.add_regressor('NSW_ROOFTOP_SOLAR', mode='additive')
m_nsw.add_regressor('NSW_HYDRO', mode='additive')
m_nsw.add_regressor('NSW_PUMPS', mode='additive')
m_nsw.add_regressor('NSW_DISTILLATE', mode='additive')
m_nsw.add_regressor('NSW_GAS_CCGT', mode='additive')
m_nsw.add_regressor('NSW_GAS_OCGT', mode='additive')
m_nsw.add_regressor('NSW_WIND', mode='additive')
m_nsw.add_regressor('NSW_TEMPERATURE', mode='additive')
forecast_nsw = m_nsw.fit(NSW_data).predict(NSW_data)
m_nsw.plot(forecast_nsw)



In [ ]:

NSW_data['y'].plot(figsize=(20,5))
forecast_nsw['yhat'].plot()

In [ ]:
np.exp(NSW_data.y).plot()
np.exp(forecast_nsw['yhat']).plot(figsize=(20,5))
plt.ylim(0,400)


In [ ]:
m_nsw.plot_components(forecast_nsw);

In [ ]:

((np.exp(forecast_nsw['yhat']))*1).hist(range=(0,300), figsize = (10,6))

In [ ]:
((np.exp(NSW_data.y))*1).hist(range=(0,300), figsize = (10,6))

## Test with unseen data
This is to test the first model (m) with a datset that the model has not seen before. This is data from the past 7 days for Victoria. Thus not only has the data not been seen, it also disconnected from the timeseries!


In [ ]:
VIC_test_data = pd.read_csv('./VictoriaNOV2019.csv')


In [ ]:
VIC_test_data['ds'] = pd.to_datetime(VIC_test_data['ds']) 
VIC_test_data.dropna(inplace = True)
VIC_test_data['y'] = np.log1p(VIC_test_data['y'])

In [ ]:
VIC_test_data


In [ ]:
forecast_test = m.predict(VIC_test_data)

In [ ]:
VIC_test_data.y.plot()
forecast_test['yhat'].plot()


In [ ]:
np.exp(VIC_test_data.y).plot()
np.exp(forecast_test['yhat']).plot()

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
pcorr, _ = pearsonr((VIC_test_data.y),(forecast_test['yhat'])) 

print("Pearson Corr:", pcorr)
scorr, _ = spearmanr((VIC_test_data.y),(forecast_test['yhat']))
print("Spearman Corr:", scorr)
kcorr, _ = kendalltau((VIC_test_data.y),(forecast_test['yhat']))
print("Kendall tau Corr:", kcorr)


### I think it might need more work! :)

Let's see if the New South Wales model is better.


In [ ]:
NSW_test_data = pd.read_csv('./NSW_test_dataNOV2019.csv')
NSW_test_data['ds'] = pd.to_datetime(NSW_test_data['ds']) 
NSW_test_data.dropna(inplace = True)
NSW_test_data['y'] = np.log1p(NSW_test_data['y'])

In [ ]:
forecast_test_nsw = m_nsw.predict(NSW_test_data)

In [ ]:
np.exp(NSW_test_data.y).plot(figsize = (16,8), y = NSW_test_data.ds)
(np.exp(forecast_test_nsw['yhat'])).plot(figsize = (16,8))


In [ ]:
 NSW_test_data.set_index('ds')

In [ ]:
from scipy.stats import pearsonr, spearmanr

In [ ]:
plt.scatter(np.exp(NSW_test_data.y),(np.exp(forecast_test_nsw['yhat'])))

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
pcorr, _ = pearsonr((NSW_test_data.y),(forecast_test_nsw['yhat'])) 
print("Pearson Corr:", pcorr)
scorr, _ = spearmanr((NSW_test_data.y),(forecast_test_nsw['yhat']))
print("Spearman Corr:", scorr)
kcorr, _ = kendalltau((NSW_test_data.y),(forecast_test_nsw['yhat']))
print("Kendall tau Corr:", kcorr)

In [ ]:
NSW_data_demand = NSW_data
NSW_data_demand['PRICE'] = NSW_data['y']
NSW_data_demand['y'] = NSW_data["NSW_DEMAND"]

In [ ]:

m_nsw_demand = Prophet()
forecast_nsw_demand = m_nsw_demand.fit(NSW_data_demand).predict(NSW_data)
m_nsw_demand.plot(forecast_nsw_demand);


In [ ]:

m_nsw_demand.plot(forecast_nsw_demand);


In [ ]:
NSW_test_data["charge"] =  (np.exp(NSW_test_data.y)<50)*1
forecast_test_nsw["charge"] = (np.exp(forecast_test_nsw['yhat'])<50).astype(int)

In [ ]:
NSW_test_data['charge'].plot()
forecast_test_nsw["charge"].plot()

In [ ]:
import statsmodels.api as sm
sm.graphics.tsa.plot_acf(NSW_data['y'], lags=350, alpha=0.2)

sm.graphics.tsa.plot_pacf(NSW_data['y'], lags=350, alpha=0.2)


In [ ]:
sm.graphics.tsa.plot_acf(VIC_data['VIC_PRICE'], lags=350, alpha=0.2);

sm.graphics.tsa.plot_pacf(VIC_data['VIC_PRICE'], lags=350, alpha=0.2);


## Now for a bit more 

In [ ]:
NSW_data['y'] = NSW_data_demand['PRICE'] 

In [ ]:
NSW_data[['y']]



In [ ]:
forecast_test_nsw

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
tscv = TimeSeriesSplit(max_train_size=48*30*30,n_splits=2)

In [ ]:
NSW_data['y'].shape

In [ ]:
21936//672

In [ ]:
train_df=NSW_data.reset_index(drop=True)
X_train = pd.DataFrame()
X_test = pd.DataFrame()

model = Prophet()

for train_index,test_index in tscv.split(train_df):
    
    print(train_df.iloc[train_index].shape,train_df.iloc[test_index].shape)
    X_train[['y','ds']] = train_df.iloc[train_index][['y','ds']]
    X_test[['y','ds']] = train_df.iloc[test_index][['y','ds']]
    X_train_predict = model.fit(X_train).predict(X_train)
    
    
    future = model.make_future_dataframe(periods=40)
    forecast = model.predict(future)
    break
#     print(X_test.columns)
    

In [ ]:
from fbprophet.plot import plot_forecast_component,plot,add_changepoints_to_plot

ax =model.plot(forecast)
add_changepoints_to_plot(ax.gca(),model,forecast)
# (forecast)
# plt.plot(X_test['y'])

In [ ]:
import holidays


In [ ]:
au_holidays = holidays.CountryHoliday('Australia',prov='NSW')

In [ ]:
# from datetime import date,datetime
from datetime import datetime

In [ ]:
train_timestamps = pd.to_datetime(X_train['ds'])
(train_timestamps[0].date(),train_timestamps.iloc[-1].date())
# ,X_train['ds'][-1])

In [ ]:
#[date for date in train_timestamps if date.date() in au_holidays]

In [ ]:
pd.plotting.register_matplotlib_converters()


# forecast_ts = forecast.set_index('ds')
# forecast_ts['yhat_actual'] = X_test.set_index('ds')
# X_train[['ds']]
X_test.ds = pd.to_datetime(X_test.ds)

In [ ]:
forecast_total = forecast.merge(X_test.rename(columns={'y':'yhat_actual'}),on='ds',how='left')

In [ ]:
forecast_total.set_index('ds')[['yhat','yhat_actual']].plot()

In [ ]:
X_test.shape

In [ ]:
future.shape


## OHLC Plotting

In [ ]:
NSW_data.index = pd.to_datetime(NSW_data.index)
NSW_data_OHLC = np.exp(NSW_data["y"]).resample('D').ohlc()

#daily_mean["VIC_PRICE"] = data["VIC_PRICE"].resample('D').mean()

In [ ]:
NSW_data_OHLC

In [ ]:
import plotly.graph_objects as go



In [ ]:
fig = go.Figure(data=[go.Candlestick(x=NSW_data_OHLC.index,
                open=NSW_data_OHLC['open'],
                high=NSW_data_OHLC['high'],
                low=NSW_data_OHLC['low'],
                close=NSW_data_OHLC['close'])])

fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Ohlc(x=NSW_data_OHLC.index,
                open=NSW_data_OHLC['open'],
                high=NSW_data_OHLC['high'],
                low=NSW_data_OHLC['low'],
                close=NSW_data_OHLC['close'])])

fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
VIC_data.index = pd.to_datetime(VIC_data.index)
VIC_data_OHLC = np.exp(VIC_data["y"]).resample('3H').ohlc()

In [ ]:
fig = go.Figure(data=[go.Ohlc(x=VIC_data_OHLC.index,
                open=VIC_data_OHLC['open'],
                high=VIC_data_OHLC['high'],
                low=VIC_data_OHLC['low'],
                close=VIC_data_OHLC['close'])])

fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
nsw_low = Prophet()

NSW_data_OHLC['y'] = NSW_data_OHLC['low']
NSW_data_OHLC['ds'] = NSW_data_OHLC.index

nsw_low.fit(NSW_data_OHLC)

future = nsw_low.make_future_dataframe(periods=40)
forecast_low = nsw_low.predict(future)
nsw_low.plot(forecast_low) ;

In [ ]:
nsw_high = Prophet()

NSW_data_OHLC['y'] = NSW_data_OHLC['high']
NSW_data_OHLC['ds'] = NSW_data_OHLC.index

nsw_high.fit(NSW_data_OHLC)

future = nsw_high.make_future_dataframe(periods=40)
forecast_high = nsw_high.predict(future)
nsw_high.plot(forecast_high) ;

In [ ]:
nsw_close = Prophet()

NSW_data_OHLC['y'] = NSW_data_OHLC['close']
NSW_data_OHLC['ds'] = NSW_data_OHLC.index

nsw_close.fit(NSW_data_OHLC)

future = nsw_close.make_future_dataframe(periods=40)
forecast_close = nsw_close.predict(future)
nsw_close.plot(forecast_close) ;
#forecast_total = forecast.merge(X_test.rename(columns={'y':'yhat_actual'}),on='ds',how='left')

In [ ]:
nsw_open = Prophet()

NSW_data_OHLC['y'] = NSW_data_OHLC['open']
NSW_data_OHLC['ds'] = NSW_data_OHLC.index

nsw_open.fit(NSW_data_OHLC)

future = nsw_open.make_future_dataframe(periods=40)
forecast_open = nsw_open.predict(future)
nsw_open.plot(forecast_open) ;

In [ ]:
fig = go.Figure(data=[go.Ohlc(x=forecast_high.ds,
                open=forecast_open['yhat'],
                high=forecast_high['yhat'],
                low=forecast_low['yhat'],
                close=forecast_close['yhat'])])

fig2 = go.Figure(data=[go.Ohlc(x=NSW_data_OHLC.index,
                open=NSW_data_OHLC['open'],
                high=NSW_data_OHLC['high'],
                low=NSW_data_OHLC['low'],
                close=NSW_data_OHLC['close'])])

fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()
fig2.update(layout_xaxis_rangeslider_visible=False)
fig2.show()

Nope, that is rubbish

## Good Version of OHLC and Candlesticks


In [ ]:
forecast_nsw.index = forecast_nsw.ds
NSW_forecast_ohlc = np.exp(forecast_nsw['yhat']).resample('6H').ohlc()


In [ ]:
fig = go.Figure(data=[go.Ohlc(x=NSW_forecast_ohlc.index,
                open=NSW_forecast_ohlc['open'],
                high=NSW_forecast_ohlc['high'],
                low=NSW_forecast_ohlc['low'],
                close=NSW_forecast_ohlc['close'])])

fig2 = go.Figure(data=[go.Ohlc(x=NSW_data_OHLC.index,
                open=NSW_data_OHLC['open'],
                high=NSW_data_OHLC['high'],
                low=NSW_data_OHLC['low'],
                close=NSW_data_OHLC['close'])])

fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()
fig2.update(layout_xaxis_rangeslider_visible=False)
fig2.show()

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=NSW_forecast_ohlc.index,
                open=NSW_forecast_ohlc['open'],
                high=NSW_forecast_ohlc['high'],
                low=NSW_forecast_ohlc['low'],
                close=NSW_forecast_ohlc['close'],
                increasing_line_color= 'lightgreen', decreasing_line_color= 'pink')])

fig.update_layout(
    title='Forecast New South Wales Price',
    yaxis_title='Price per MWHr')


fig2 = go.Figure(data=[go.Candlestick(x=NSW_data_OHLC.index,
                open=NSW_data_OHLC['open'],
                high=NSW_data_OHLC['high'],
                low=NSW_data_OHLC['low'],
                close=NSW_data_OHLC['close'])])

fig2.update_layout(
    title='Actual New South Wales Price',
    yaxis_title='Price per MWHr')

fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()
fig2.update(layout_xaxis_rangeslider_visible=False)
fig2.show()

In [ ]:
fig_vic = go.Figure(data=[go.Candlestick(x=VIC_forecast_ohlc.index,
                open=VIC_forecast_ohlc['open'],
                high=VIC_forecast_ohlc['high'],
                low=VIC_forecast_ohlc['low'],
                close=VIC_forecast_ohlc['close'],
                increasing_line_color= 'lightgreen', decreasing_line_color= 'pink')])

fig_vic.update_layout(
    title='Forecast Victorian Price',
    yaxis_title='Price per MWHr')

fig2_vic = go.Figure(data=[go.Candlestick(x=VIC_data_OHLC.index,
                open=VIC_data_OHLC['open'],
                high=VIC_data_OHLC['high'],
                low=VIC_data_OHLC['low'],
                close=VIC_data_OHLC['close'])])

fig2_vic.update_layout(
    title='Actual Victorian Price',
    yaxis_title='Price per MWHr')

fig_vic.update(layout_xaxis_rangeslider_visible=False)
fig_vic.show()
fig2_vic.update(layout_xaxis_rangeslider_visible=False)
fig2_vic.show()

In [ ]:
import plotly.express as px


fig_test = go.Figure(data=go.Scatter(x=NSW_test_data.ds, y=np.exp(NSW_test_data.y),name='Actual Price'))
fig_test.add_trace(go.Scatter(x=NSW_test_data.ds, y=np.exp(forecast_test_nsw['yhat']),
                    mode='lines',
                    name='Modelled'))
fig_test.update_layout(title='NSW Wholesale Price for the past two weeks',
                   yaxis_title='Price per MWHr')
fig_test.show()

In [ ]:
forecast_test_nsw.index = forecast_test_nsw.ds
NSW_test_ohlc = np.exp(forecast_test_nsw['yhat']).resample('3H').ohlc()

In [ ]:
fig_test_nsw = go.Figure(data=[go.Candlestick(x=NSW_test_ohlc.index,
                open=NSW_test_ohlc['open'],
                high=NSW_test_ohlc['high'],
                low=NSW_test_ohlc['low'],
                close=NSW_test_ohlc['close'], showlegend = False)])
fig_test_nsw.update_layout(title='NSW Wholesale Price for the past two weeks',
                   yaxis_title='Price per MWHr')
fig_test_nsw.add_trace(go.Scatter(x=NSW_test_data.ds, y=np.exp(NSW_test_data.y), name='Actual Price', 
                                  line=dict(color="black",width=1, dash='dot'), showlegend = False))
fig_test_nsw.update(layout_xaxis_rangeslider_visible=False)
fig_test_nsw.show()